[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# Aplicación ilustrativa.

El directorio ```src/api_basica```contiene una estructura básica de una aplicación de *FastAPI*.

```
├── crud.py
├── data
│   └── __init__.py
│       
├── database.py
├── db.sqlite3
├── main.py
├── models.py
├── requirements.txt
└── schemas.py
```

### El script ```database.py```.

Este *script* define las conexiones y los objetos de *SQLAlchemy*.

``` python
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


SQLALCHEMY_DATABASE_URL = "sqlite:///db.sqlite3"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()
```

In [ ]:
%pycat src/api_basica/database.py

### El *script* ```models.py```.

Este script contiene las definiciones de modelos creados a partir de ```models.Base```.
En esteb caso, sólo contiene la definición de la clase ```Alumno```.

``` python
from sqlalchemy import Column, Integer, String, Float, Boolean
from database import Base

class Alumno(Base):
    '''Modelo de alumnos.'''
    __tablename__ = 'alumnos'
    cuenta = Column(Integer, primary_key=True)
    nombre = Column(String(50))
    primer_apellido = Column(String(50))
    segundo_apellido = Column(String(50))
    carrera = Column(String(50))
    semestre = Column(Integer)
    promedio = Column(Float)
    al_corriente = Column(Boolean)
```

In [ ]:
%pycat src/api_basica/models.py

### El *script* ```schemas.py```.

Este *script* contiene los objetos instaciandos de *Pydantic* que serán usados para validar los datos de entrada (```SchemaAlumnoIn```) y salida (```SchemaAlumno```).

``` python
from pydantic import BaseModel, Field, PositiveInt
from typing import Optional, Literal
from data import CARRERAS


class SchemaAlumno(BaseModel):
    cuenta: int
    nombre: str
    primer_apellido: str
    segundo_apellido: str = ''
    carrera: Literal[tuple(CARRERAS)]
    semestre: PositiveInt
    promedio: float = Field(ge=0, le=10)
    al_corriente: bool
        
    class Config:
        orm_mode = True
        

class SchemaAlumnoIn(BaseModel):
    nombre: str
    primer_apellido: str
    segundo_apellido: str = ''
    carrera: Literal[tuple(CARRERAS)]
    semestre: PositiveInt
    promedio: float = Field(ge=0, le=10)
    al_corriente: bool
        
    class Config:
        orm_mode = True
```

In [ ]:
%pycat src/api_basica/schemas.py

### El *script* ```crud.py```.

Este *script* contine las funciones que van a realizar las operaciones *CRUD* con los modelos y los datos que ingresan como argumentos.

``` python
from sqlalchemy.orm import Session

import models
import schemas


def consulta_alumnos(db: Session, skip: int = 0, limit: int = 100):
    return db.query(models.Alumno).offset(skip).limit(limit).all()


def consulta_alumno(db: Session, cuenta: int):
    return db.query(models.Alumno).filter(models.Alumno.cuenta == cuenta).first()


def alta_alumno(db: Session, cuenta: int, candidato: schemas.SchemaAlumnoIn):
    alumno = models.Alumno(cuenta=cuenta, **dict(candidato))
    db.add(alumno)
    db.commit()
    db.refresh(alumno)
    return alumno


def baja_alumno(db: Session, alumno: models.Alumno):
    db.delete(alumno)
    db.commit()
    return True
```

In [ ]:
%pycat src/api_basica/crud.py

### El *script* ```main.py```.

Este *script* crea una aplicación ```app``` a partir de la clase ```fastapi.FastAPI``` e implementa los métodos y rutas de los *end points* de la *API*.

```python
from typing import List
from fastapi import FastAPI, Depends
from fastapi.exceptions import HTTPException
from sqlalchemy.orm import Session
import crud
import models
import schemas
from database import SessionLocal, engine

models.Base.metadata.create_all(bind=engine)

app = FastAPI()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


@app.get("/api/", response_model=List[schemas.SchemaAlumno])
def vuelca_base(skip: int = 0, limit: int = 100, db: Session = Depends(get_db)):
    alumnos = crud.consulta_alumnos(db, skip=skip, limit=limit)
    return alumnos


@app.get("/api/{cuenta}", response_model=schemas.SchemaAlumno)
def get_alumno(cuenta, db: Session = Depends(get_db)):
    alumno = crud.consulta_alumno(db=db, cuenta=cuenta)
    if alumno:
        return alumno
    else:
        raise HTTPException(status_code=404, detail="Recurso no encontrado")

        
@app.delete("/api/{cuenta}")
def delete_alumno(cuenta, db: Session = Depends(get_db)):
    alumno = crud.consulta_alumno(db=db, cuenta=cuenta)
    if alumno:
        crud.baja_alumno(db=db, alumno=alumno)
        return {}
    else:
        raise HTTPException(status_code=404, detail="Recurso no encontrado")

        
@app.post("/api/{cuenta}", response_model=schemas.SchemaAlumno)
def post_alumno(cuenta, candidato: schemas.SchemaAlumnoIn, db: Session = Depends(get_db)):
    alumno = crud.consulta_alumno(db=db, cuenta=cuenta)
    if alumno:
        raise HTTPException(status_code=409, detail="Recurso existente")
    return crud.alta_alumno(db=db, cuenta=cuenta, candidato=candidato)        
        
        
@app.put("/api/{cuenta}", response_model=schemas.SchemaAlumno)
def put_alumno(cuenta, candidato: schemas.SchemaAlumnoIn, db: Session = Depends(get_db)):
    alumno = crud.consulta_alumno(db=db, cuenta=cuenta)
    if alumno:
        crud.delete_alumno(db=db, alumno=alumno)
        return crud.alta_alumno(db=db, cuenta=cuenta, candidato=candidato)
    else:
        raise HTTPException(status_code=404, detail="Recurso no encontrado")
```

In [ ]:
%pycat src/api_basica/main.py

## Ejecución.

In [ ]:
%cd src/api_basica/

In [ ]:
!uvicorn main:app --host 0.0.0.0 --reload

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>